In [ ]:
# TODO: requirements.txt
# pip install matplotlib
# pip install numpy
# pip install nbconvert
#   (for git)

In [ ]:
from typing import Callable

# Import
import os
from pupil_info import *
import matplotlib.pyplot as plt
import numpy as np
from notes_file import *
from typing import Callable
from matplotlib.axes import Axes
from matplotlib.figure import Figure
import matplotlib.font_manager as font_manager
import matplotlib.patches as mpatches

# Import local_features (hack)
import sys
import os
sys.path.append(os.path.abspath(".."))
from local_features import *
from util import *


INFO: list[PupilInfo] = get_all_pupil_info()


In [ ]:
# Set Heuristica as font

font_path = os.path.join(os.path.abspath('..'), 'font.otf')
assert os.path.exists(font_path)
font_manager.fontManager.addfont(font_path)
prop = font_manager.FontProperties(fname=font_path)

plt.rcParams['font.family'] = prop.get_name()
plt.rcParams['font.size'] = 12

In [ ]:
# Piano roll for a specified performance
from recording import OutputName

performance: PerformanceInfo = INFO[2].sessions[0].performances[2]
t = SONG_NAMES[performance.song] + " with " + ALGORITHM_NAMES[performance.algorithm]
recording = performance.recording
midi_notes: list[MidiNote] = all_notes(recording.measures)

colors = ['#DBD300', '#008FEF']

fig: Figure
ax: Axes
fig, ax = plt.subplots(figsize=(20, 5))

# Plot each note with the corresponding color
for note in midi_notes:
    color = colors[0] if note.output_name == OutputName.LOOPBACK else colors[1]
    ax.broken_barh([(note.time, note.length)], (note.note - 0.4, 0.8), facecolors=color)

# Formatting
ax.set_title(f"Piano roll of a performance ({t})")
ax.set_xticks(np.arange(0, 68, 4))
ax.set_yticks([])

ax.grid(True)

# Add legend
legend_patches = [
    plt.Line2D([0], [0], color=colors[0], lw=4, label="Pupil"),
    plt.Line2D([0], [0], color=colors[1], lw=4, label="Algorithm")
]
ax.legend(handles=legend_patches, loc="upper right")

plt.savefig('charts/0_piano_roll.svg')

In [ ]:
# Are some songs simply more fun than others?

# Evaluate this specified question
question_index: int = 3

colors = ['#B41F2C', '#B4B31F', '#501FB4']

# Get all performances
all_performances: list[PerformanceInfo] = [
    INFO[pupil_index].sessions[session_index].performances[performance_index]
    for pupil_index in range(NUM_PUPILS)
    for session_index in range(NUM_SESSIONS)
    for performance_index in range(NUM_PERFORMANCES)
]

# Keep track of all reports
all_reports: list[list[int]] = []

song_index: int
for song_index in range(NUM_SONGS):
    # Get all reports
    reports: list[int] = [performance.scores[question_index] for performance in all_performances if performance.song == song_index]

    # Append to plotting variable
    all_reports.append(reports)

fig: Figure
ax: Axes
fig, ax = plt.subplots()
plots = ax.violinplot(all_reports, showmeans=True)
ax.set_title('"I enjoyed playing this song with the algorithm", per song')

# Set colors
for pc, color in zip(plots['bodies'], colors):
    pc.set_facecolor(color)
plots['cmeans'].set_colors(colors)
plots['cmins'].set_colors(colors)
plots['cmaxes'].set_colors(colors)
plots['cbars'].set_colors(colors)

ax.yaxis.grid(True)
ax.set_ylim([0.8, 5.2])
_ = ax.set_xticks([y + 1 for y in range(NUM_SONGS)], labels=list(SONG_IDS.keys()))
ax.set_yticks(range(1, 6))

ax.set_ylabel("Likert score")

plt.savefig('charts/2_song_enjoyability.svg')

In [ ]:
# Reciprocity scores

# Evaluate this specified question and algorithm


colors = ['#1F77B4', '#B45C1F', '#1FB42A']

# Get all performances
all_performances: list[PerformanceInfo] = [
    INFO[pupil_index].sessions[session_index].performances[performance_index]
    for pupil_index in range(NUM_PUPILS)
    for session_index in range(NUM_SESSIONS)
    for performance_index in range(NUM_PERFORMANCES)
]

all_reports: list[list[int]] = []

question_index: int = 0
algorithm_index: int
for algorithm_index in range(NUM_ALGORITHMS):
    # Get reports of performances of algorithm
    reports: list[int] = [
        performance.scores[question_index]
        for performance in all_performances
        if performance.algorithm == algorithm_index
    ]

    all_reports.append(reports)

question_index: int = 1
algorithm_index: int
for algorithm_index in range(NUM_ALGORITHMS):
    # Get reports of performances of algorithm
    reports: list[int] = [
        performance.scores[question_index]
        for performance in all_performances
        if performance.algorithm == algorithm_index
    ]

    all_reports.append(reports)

fig: Figure
ax: Axes
fig, ax = plt.subplots(figsize=(10, 5))
plots = ax.violinplot(all_reports, showmeans=True)
ax.set_title('Reciprocity ratings, per algorithm')

# Set colors
for pc, color in zip(plots['bodies'], colors * 2):
    pc.set_facecolor(color)
plots['cmeans'].set_colors(colors)
plots['cmins'].set_colors(colors)
plots['cmaxes'].set_colors(colors)
plots['cbars'].set_colors(colors)

ax.yaxis.grid(True)
ax.set_ylim([0.8, 5.2])
_ = ax.set_xticks([y + 1 for y in range(NUM_ALGORITHMS * 2)], labels=["NR", "TFOv1\nAlgorithm-to-pupil", "TMv1", "NR", "TFOv1\nPupil-to-algorithm", "TMv1"])
ax.set_yticks(range(1, 6))

ax.set_ylabel("Likert score")

print([np.average(s) for s in all_reports])

plt.savefig('charts/14_algorithm_reciprocity.svg')

In [ ]:
# Are some algorithms simply more fun than others?

# Evaluate this specified question and algorithm
question_index: int = 3

colors = ['#1F77B4', '#B45C1F', '#1FB42A']

# Get all performances
all_performances: list[PerformanceInfo] = [
    INFO[pupil_index].sessions[session_index].performances[performance_index]
    for pupil_index in range(NUM_PUPILS)
    for session_index in range(NUM_SESSIONS)
    for performance_index in range(NUM_PERFORMANCES)
]

all_reports: list[list[int]] = []

algorithm_index: int
for algorithm_index in range(NUM_ALGORITHMS):
    # Get reports of performances of algorithm
    reports: list[int] = [
        performance.scores[question_index]
        for performance in all_performances
        if performance.algorithm == algorithm_index
    ]

    all_reports.append(reports)

fig: Figure
ax: Axes
fig, ax = plt.subplots()
plots = ax.violinplot(all_reports, showmeans=True)
ax.set_title('"I enjoyed playing this song with the algorithm", per algorithm')

# Set colors
for pc, color in zip(plots['bodies'], colors):
    pc.set_facecolor(color)
plots['cmeans'].set_colors(colors)
plots['cmins'].set_colors(colors)
plots['cmaxes'].set_colors(colors)
plots['cbars'].set_colors(colors)

ax.yaxis.grid(True)
ax.set_ylim([0.8, 5.2])
_ = ax.set_xticks([y + 1 for y in range(NUM_ALGORITHMS)], labels=list(ALGORITHM_IDS.keys()))
ax.set_yticks(range(1, 6))

ax.set_ylabel("Likert score")

print([np.average(s) for s in all_reports])

plt.savefig('charts/3_algorithm_enjoyability.svg')

In [ ]:
# Enjoyability per session

# Evaluate this specified question and algorithm
question_index: int = 3

colors = ['#008FEF', '#42B3FF', '#7AC9FF', "#A3DAFF"]

all_reports: list[list[int]] = []

session_index: int
for session_index in range(NUM_SESSIONS):
    performances = [
        INFO[pupil_index].sessions[session_index].performances[performance_index]
        for pupil_index in range(NUM_PUPILS)
        for performance_index in range(NUM_PERFORMANCES)
    ]

    # Get reports of performances of algorithm
    reports: list[int] = [
        performance.scores[question_index]
        for performance in performances
    ]

    all_reports.append(reports)

fig: Figure
ax: Axes
fig, ax = plt.subplots()
plots = ax.violinplot(all_reports, showmeans=True)
ax.set_title('"I enjoyed playing this song with the algorithm", per session')

# Set colors
for pc, color in zip(plots['bodies'], colors):
    pc.set_facecolor(color)
plots['cmeans'].set_colors(colors)
plots['cmins'].set_colors(colors)
plots['cmaxes'].set_colors(colors)
plots['cbars'].set_colors(colors)

ax.yaxis.grid(True)
ax.set_ylim([0.8, 5.2])
_ = ax.set_xticks([y + 1 for y in range(NUM_SESSIONS)], labels=[f"Session {i + 1}" for i in range(NUM_SESSIONS)])
ax.set_yticks(range(1, 6))

ax.set_ylabel("Likert score")

print([np.average(s) for s in all_reports])

plt.savefig('charts/4_session_enjoyability.svg')

In [ ]:
# Get all performances
all_performances: list[PerformanceInfo] = [
    INFO[pupil_index].sessions[session_index].performances[performance_index]
    for pupil_index in range(NUM_PUPILS)
    for session_index in range(NUM_SESSIONS)
    for performance_index in range(NUM_PERFORMANCES)
]

fig, axs = plt.subplots(NUM_QUESTIONS, figsize=(8, 10))

colors = ['#1F77B4', '#B45C1F', '#1FB42A']

questions = [
    "I feel that the algorithm could respond well to me",
    "I feel that I could respond well to the algorithm",
    "I feel that the algortihm gives me inspiration",
    "I enjoyed playing this song with the algorithm",
]

fig.subplots_adjust(hspace=0.5)

width = 0.25

for algorithm_index in range(NUM_ALGORITHMS):
    # Get scores of performances of algorithm
    performances: list[PerformanceInfo] = [
        performance
        for performance in all_performances
        if performance.algorithm == algorithm_index
    ]

    for q in range(NUM_QUESTIONS):
        q_scores: list[int] = [sum([p.scores[q] == v for p in performances]) for v in range(1, 6)]
        axs[q].title.set_text(f'"{questions[q]}"')
        axs[q].bar(x=np.array(range(1, 6)) + width * algorithm_index,height=q_scores, width=width, color=colors[algorithm_index])
        axs[q].set_yticks(range(0, 15, 5))

legend_patches = [
    mpatches.Patch(color=colors[algorithm_index], label=ALGORITHM_NAMES[algorithm_index]) for algorithm_index in range(NUM_ALGORITHMS)
]
fig.legend(handles=legend_patches, loc="outside upper right")

plt.savefig('charts/response_scores.png')

In [ ]:
# Are pupils self-reporting feeling more musical over time?
# Are pupils self-reporting feeling more response (from themselves; from the algorithm) over time?
# Are pupils self-reporting feeling more enjoyment over time?

# Evaluate this specified question
# 0: I feel that the algorithm could respond well to me.
# 1: I feel that I could respond well to the algorithm.
# 2: I feel that the algorithm gives me inspiration.
# 3: I enjoyed playing this song with the algorithm.
question_index: int = 3

session_index: int
for session_index in range(NUM_SESSIONS):
    # Get all performances of the session
    performances: list[PerformanceInfo] = [
        INFO[pupil_index].sessions[session_index].performances[performance_index]
        for pupil_index in range(NUM_PUPILS)
        for performance_index in range(NUM_PERFORMANCES)
    ]

    # Get the question reports
    reports: list[int] = [
        performance.scores[question_index]
        for performance in performances
    ]

    # (TODO: remove) Filter all not-yet-filled-in answers
    reports = [score for score in reports if score > 0]

    print(session_index, ": ", np.average(reports), sep='')


In [ ]:
# For a specific algorithm, for a specific metric, how does it (and its distribution) evolve over the sessions?

# Evaluate this specific metric
metric: Callable[[list[Measure]], float]
metric = melodic_arc_duration_avg

# Evaluate this specific algorithm
algorithm_index: int = 0

all_metrics: list[list[float]] = []

session_index: int
for session_index in range(NUM_SESSIONS):
    # Get performances
    recordings: list[Recording] = [
        INFO[pupil_index].sessions[session_index].performances[performance_index].recording
        for pupil_index in range(NUM_PUPILS)
        for performance_index in range(NUM_PERFORMANCES)
    ]

    # Get metrics
    metrics: list[float] = [
        metric(four, recording.bpm)
        for recording in recordings
        for four in recording.fours
    ]

    all_metrics.append(metrics)

fig: Figure
ax: Axes
fig, ax = plt.subplots()
ax.violinplot(all_metrics, showmeans=True)
ax.set_title('Violin plot')

ax.yaxis.grid(True)

ax.set_xticks([1, 2, 3])

plt.show()

In [ ]:
# (out of personal interest) Is the fifth really more popular in solos of Summertime?

from functools import partial

all_metrics: list[list[float]] = []

for song_index in range(NUM_SONGS):
    fifth = SONG_PITCH_CLASSES[SONG_NAMES[song_index]][2]

    # Get performances
    performances = [
        INFO[pupil_index].sessions[session_index].performances[performance_index]
        for pupil_index in range(NUM_PUPILS)
        for session_index in range(NUM_SESSIONS)
        for performance_index in range(NUM_PERFORMANCES)
    ]

    # Filter by song
    performances = [
        performance
        for performance in performances
        if performance.song == song_index
    ]

    # Get all recordings
    recordings: list[Recording] = [performance.recording for performance in performances]

    # Get metrics
    metrics: list[float] = [
        note_share(four, pitch_class=fifth)
        for recording in recordings
        for four in recording.fours
    ]

    all_metrics.append(metrics)


fig: Figure
ax: Axes
fig, ax = plt.subplots()
ax.violinplot(all_metrics, showmeans=True)
ax.set_title('Violin plot')

ax.yaxis.grid(True)
_ = ax.set_xticks([y + 1 for y in range(NUM_SONGS)], labels=list(SONG_IDS.keys()))

plt.show()

In [ ]:
# Metrics over fours (dot plot)

fig, axs = plt.subplots(NUM_ALGORITHMS, figsize=(8, 10))
fig.subplots_adjust(hspace=0.5)

colors = ['#DBD300', '#008FEF']

width = 0.166

ticks: list[int]

# Get all performances
all_performances: list[PerformanceInfo] = [
    INFO[pupil_index].sessions[session_index].performances[performance_index]
    for pupil_index in range(NUM_PUPILS)
    for session_index in range(NUM_SESSIONS)
    for performance_index in range(NUM_PERFORMANCES)
]

algorithm_index: int
for algorithm_index in range(NUM_ALGORITHMS):
    # Get performances of algorithm
    performances: list[PerformanceInfo] = [
        performance
        for performance in all_performances
        if performance.algorithm == algorithm_index
    ]

    metrics = make_average_list(64 // 4)

    # Get metrics of all fours
    for performance in performances:
        for i, four in enumerate(performance.recording.fours):
            if i >= (64 // 4): continue

            m = extrema_ratio(four, SONG_BPMS[SONG_NAMES[performance.song]])
            ylabel = "Extrema ratio"
            # m = note_share_135(four, SONG_PITCH_CLASSES[SONG_NAMES[performance.song]])
            # ylabel = "Root/third/fifth note share"
            ticks = list(range(2))

            # m = npvi(four, SONG_BPMS[SONG_NAMES[performance.song]])
            # ticks = list(range(0, 100, 20))
            # ylabel = "NPVI"

            metrics[i].add(m)

    avg_metrics = get_average(metrics)

    x = list(range(len(metrics)))
    c = [colors[i % 2] for i in x]

    ax = axs[algorithm_index]

    ax.set_title(ALGORITHM_NAMES[algorithm_index])
    ax.set_xlabel("Time (in measures)")
    ax.set_ylabel(ylabel)

    ax.scatter(x, avg_metrics, c=c, marker='o')
    ax.set_yticks(ticks)

legend_patches = [
    mpatches.Patch(color=colors[i], label=["Human four", "Algorithm four"][i]) for i in range(2)
]
fig.legend(handles=legend_patches, loc="outside upper right")

plt.savefig('charts/5_metrics_extrema_ratio.svg')

In [ ]:
# How does every four relate to the next, on average?

colors = ['#B41F2C', '#B4B31F', '#501FB4']
width = 0.25

fig: Figure
ax: Axes
fig, ax = plt.subplots(layout='constrained')

ax.title.set_text("Token edit distance per song")
ax.set_xticklabels(["algorithm-to-pupil", "pupil-to-algorithm", "average"])
ax.set_xticks(np.array(range(3)) + width)
# ax.set_yticks(range(0, 120, 20))

song_index: int
for song_index in range(NUM_SONGS):
    print(f"song {song_index}")

    # Get all performances
    all_performances: list[PerformanceInfo] = [
        INFO[pupil_index].sessions[session_index].performances[performance_index]
        for pupil_index in range(NUM_PUPILS)
        for session_index in range(NUM_SESSIONS)
        for performance_index in range(NUM_PERFORMANCES)
    ]

    performances = [p for p in all_performances if p.song == song_index]

    edit_distances = np.zeros(15)

    for performance in performances:
        edit_distances += performance.edit_distances

    edit_distances /= len(performances)

    height = [
        np.average(edit_distances[::2]), # Algorithm to pupil
        np.average(edit_distances[1::2]), # Pupil to algorithm
        np.average(edit_distances), # Both ways
    ]

    stds = [
        np.std(edit_distances[::2]), # Algorithm to pupil
        np.std(edit_distances[1::2]), # Pupil to algorithm
        np.std(edit_distances), # Both ways
    ]
    
    

    x = np.array(range(3)) + width * song_index
    ax.bar(x=x,height=height, width=width, color=colors[song_index])

    plt.errorbar(x, height, stds, fmt='.', color='Black', elinewidth=2,errorevery=1, alpha=0.5, capsize = 2)

# Add legend
legend_patches = [
    mpatches.Patch(color=colors[song_index], label=SONG_NAMES[song_index]) for song_index in range(NUM_SONGS)
]
fig.legend(handles=legend_patches, loc="outside upper right")

plt.savefig('charts/8_song_edit_distance.svg')

In [ ]:
# How does every four relate to the next, on average?

import matplotlib.patches as mpatches

colors = ['#1F77B4', '#B45C1F', '#1FB42A']
width = 0.25

fig: Figure
ax: Axes
fig, ax = plt.subplots(layout='constrained')

ax.title.set_text("Token edit distance per algorithm")
ax.set_xticklabels(["algorithm-to-pupil", "pupil-to-algorithm", "average"])
ax.set_xticks(np.array(range(3)) + width)
# ax.set_yticks(range(0, 120, 20))

algorithm_index: int
for algorithm_index in range(NUM_ALGORITHMS):
    print(f"algorithm {algorithm_index}")

    # Get all performances
    all_performances: list[PerformanceInfo] = [
        INFO[pupil_index].sessions[session_index].performances[performance_index]
        for pupil_index in range(NUM_PUPILS)
        for session_index in range(NUM_SESSIONS)
        for performance_index in range(NUM_PERFORMANCES)
    ]

    performances = [p for p in all_performances if p.algorithm == algorithm_index]

    edit_distances = np.zeros(15)

    for performance in performances:
        edit_distances += performance.edit_distances

    edit_distances /= len(performances)

    height = [
        np.average(edit_distances[::2]), # Algorithm to pupil
        np.average(edit_distances[1::2]), # Pupil to algorithm
        np.average(edit_distances), # Both ways
    ]

    stds = [
        np.std(edit_distances[::2]), # Algorithm to pupil
        np.std(edit_distances[1::2]), # Pupil to algorithm
        np.std(edit_distances), # Both ways
    ]
    
    

    x = np.array(range(3)) + width * algorithm_index
    ax.bar(x=x,height=height, width=width, color=colors[algorithm_index])

    plt.errorbar(x, height, stds, fmt='.', color='Black', elinewidth=2,errorevery=1, alpha=0.5, capsize = 2)

# Add legend
legend_patches = [
    mpatches.Patch(color=colors[algorithm_index], label=ALGORITHM_NAMES[algorithm_index]) for algorithm_index in range(NUM_ALGORITHMS)
]
fig.legend(handles=legend_patches, loc="outside upper right")

plt.savefig('charts/9_algorithm_edit_distance.svg')

In [ ]:
# How does every four relate to the next, on average?

import matplotlib.patches as mpatches

colors = ['#008FEF', '#42B3FF', '#7AC9FF', "#A3DAFF"]
width = 0.166

fig: Figure
ax: Axes
fig, ax = plt.subplots(layout='constrained')

ax.title.set_text("Token edit distance per session")
ax.set_xticklabels(["algorithm-to-pupil", "pupil-to-algorithm", "average"])
ax.set_xticks(np.array(range(3)) + width)
# ax.set_yticks(range(0, 120, 20))

session_index: int
for session_index in range(NUM_SESSIONS):
    # Get all performances
    performances: list[PerformanceInfo] = [
        INFO[pupil_index].sessions[session_index].performances[performance_index]
        for pupil_index in range(NUM_PUPILS)
        for performance_index in range(NUM_PERFORMANCES)
    ]

    edit_distances = np.zeros(15)

    for performance in performances:
        edit_distances += performance.edit_distances

    edit_distances /= len(performances)

    height = [
        np.average(edit_distances[::2]), # Algorithm to pupil
        np.average(edit_distances[1::2]), # Pupil to algorithm
        np.average(edit_distances), # Both ways
    ]

    stds = [
        np.std(edit_distances[::2]), # Algorithm to pupil
        np.std(edit_distances[1::2]), # Pupil to algorithm
        np.std(edit_distances), # Both ways
    ]

    x = np.array(range(3)) + width * session_index
    ax.bar(x=x,height=height, width=width, color=colors[session_index])

    plt.errorbar(x, height, stds, fmt='.', color='Black', elinewidth=2,errorevery=1, alpha=0.5, capsize = 2)

# Add legend
legend_patches = [
    mpatches.Patch(color=colors[i], label="Session " + str(i + 1)) for i in range(NUM_SESSIONS)
]
fig.legend(handles=legend_patches, loc="outside upper right")

plt.savefig('charts/10_session_edit_distance.svg')

In [ ]:
import matplotlib.patches as mpatches

def performance_interval_avg(performance: PerformanceInfo, four: list[Measure]): return interval_avg(four)
def performance_melodic_arc_duration_avg(performance: PerformanceInfo, four: list[Measure]): return melodic_arc_duration_avg(four, SONG_BPMS[SONG_NAMES[performance.song]])
def performance_npvi(performance: PerformanceInfo, four: list[Measure]): return npvi(four, SONG_BPMS[SONG_NAMES[performance.song]])
def performance_note_share_135(performance: PerformanceInfo, four: list[Measure]): return note_share_135(four, SONG_PITCH_CLASSES[SONG_NAMES[performance.song]])

features = [
    performance_interval_avg,
    performance_melodic_arc_duration_avg,
    performance_npvi,
    performance_note_share_135,
]

feature_names = [
    "Average absolute interval",
    "Average melodic arc duration",
    "nPVI",
    "Note share of root, third and fifth",
]

# Evaluate this specific algorithm
colors = ['red', 'green', 'blue']

fig: Figure
axs: list[Axes]
fig, axs = plt.subplots(2, 2, figsize=(8, 8), layout='constrained')

for feature_index in range(len(features)):
    feature: Callable = features[feature_index]

    algorithm_index: int
    for algorithm_index in range(NUM_ALGORITHMS):
        values = np.zeros(NUM_SESSIONS * NUM_PERFORMANCES)

        index: int
        for index in range(NUM_SESSIONS * NUM_PERFORMANCES):
            session_index: int = index // NUM_PERFORMANCES
            performance_index: int = index % NUM_PERFORMANCES
            
            # Get performances
            performances: list[PerformanceInfo] = [
                INFO[pupil_index].sessions[session_index].performances[performance_index]
                for pupil_index in range(NUM_PUPILS)
            ]

            # Get metrics
            new_values: list[float] = np.average([
                feature(performance, four)
                for performance in performances
                for four in performance.recording.fours
                if performance.algorithm == algorithm_index
            ])

            values[index] += new_values

        values /= NUM_SESSIONS * NUM_PERFORMANCES

        ax = axs[feature_index // 2][feature_index % 2]
        ax.set_title(feature_names[feature_index])
        ax.plot(np.array(range(NUM_SESSIONS * NUM_PERFORMANCES)) + 1, values, color=colors[algorithm_index])
        
        ax.set_xticks([1, 4, 7], labels=["1", "2", "3"])
        ax.set_xticks(range(1, 10), minor=True)
        
        ax.set_xlim(0.8, 9.2)

        ax.set_xlabel("performance")


axs[0][1].set_ylabel("t (seconds)")


# Add legend
legend_patches = [
    mpatches.Patch(color=colors[algorithm_index], label=ALGORITHM_NAMES[algorithm_index]) for algorithm_index in range(NUM_ALGORITHMS)
]
fig.legend(handles=legend_patches, loc="outside upper right")

plt.savefig('charts/global_features.png')

In [ ]:
# Self / peer correlation

self_scores = []
peer_scores = []

# Get all performances
all_performances: list[PerformanceInfo] = [
    INFO[pupil_index].sessions[session_index].performances[performance_index]
    for pupil_index in range(NUM_PUPILS)
    for session_index in range(NUM_SESSIONS)
    for performance_index in range(NUM_PERFORMANCES)
]

performance: PerformanceInfo
for performance in all_performances:

    p_self_scores = performance.scores

    others = performance.expert_scores
    
    if len(others) == 0: continue

    p_peer_scores = [float(np.average(z)) for z in zip(others[0], others[1])]

    self_scores += p_self_scores
    peer_scores += p_peer_scores



fig: Figure
ax: Axes
fig, ax = plt.subplots()
ax.set_title('Self-rating versus expert rating')


# (via https://python-graph-gallery.com/scatterplot-with-regression-fit-in-matplotlib/)
# Fit linear regression via least squares with numpy.polyfit
# It returns an slope (b) and intercept (a)
# deg=1 means linear fit (i.e. polynomial of degree 1)
b, a = np.polyfit(self_scores, peer_scores, deg=1)

# Create sequence of 100 numbers from 0 to 100
xseq = np.linspace(0.7, 5.3, num=100)

# Plot regression line
ax.plot(xseq, a + b * xseq, color="#7777FF", lw=2.5)



ax.set_xlabel("Self-rating (Likert score)")
ax.set_ylabel("Expert rating (Likert score)")

ax.set_xlim((0.5, 5.5))
ax.set_ylim((0.5, 5.5))

ax.set_xticks(range(1, 6))
ax.set_yticks(range(1, 6))

ax.scatter(self_scores, peer_scores, color="#0000ff44")

plt.savefig('charts/11_rating_scatter.svg')

In [ ]:
algorithm_avgs = make_average_list(3)

for participant_id in range(5):
    for performance_id in range(3):
        session_id = 0

        data: PerformanceInfo = INFO[participant_id].sessions[session_id].performances[performance_id]

        num_correct = sum([data.ordering[expert_id] == "first" for expert_id in range(3)])
        algorithm_id: int = data.algorithm

        algorithm_avgs[algorithm_id].add(num_correct, 3)

bars = get_average(algorithm_avgs)
bars = [v*100 for v in bars]



fig, ax = plt.subplots()

colors = ['#1F77B4', '#B45C1F', '#1FB42A']

ax.set_xticklabels(ALGORITHM_NAMES[i] for i in range(NUM_ALGORITHMS))
ax.set_xticks(np.array(range(3)))

ax.bar(range(NUM_ALGORITHMS), bars, label=colors, color=colors)

ax.set_ylabel('Correct orderings (%)')
ax.set_ylim(0, 100)
ax.set_title('Ordering correctness per algorithm')

plt.savefig('charts/12_ordering_correctness.svg')